In [3]:
import os
import csv
import pandas as pd
import numpy as np
import boto3
from IPython.display import clear_output
import ast
import pymongo as pm
from bson.objectid import ObjectId

## NEED TO RUN NOTEBOOK AS PYTHON3 ## 
## Otherwise can't connect using boto3 due to SSL issue with python2; installing packages using pip3 and python3 
## kernel solved the issue

In [40]:
#read the csv file and select target columns
fname = 'collection_csvs/cdm_run_v4objects_filtered_2020-09-01.csv'
# fname = 'collection_csvs/cdm_run_v5objects_filtered_2020-09-01.csv'
# fname = 'collection_csvs/cdm_run_v6objects_filtered_2020-09-01.csv'
# fname = 'collection_csvs/cdm_run_v7objects_filtered_2020-09-01.csv'
# fname = 'MuseumStation_AllDescriptives_14447_images_final_cdm_run_v5.csv'
version = 'cdm_run_v4'

data = pd.read_csv(fname)
data = data[['session_id', 'category', 'age','fileNameShortNew']]
data = data[~data['category'].isin(['something you love'])]
data.reset_index().drop(['index'], axis=1)
data['filename'] = data['fileNameShortNew']
data['url'] = data['filename'].apply(lambda x: 'https://kiddraw.s3.amazonaws.com/' + x)
data = data.rename(columns={'category':'class'})

# add a shuffle id and a games variable to indicate the number of times the current image is labelled
data_range = np.arange(data.shape[0])
np.random.shuffle(data_range)
data['shuffler_ind'] = data_range
data['games'] = '[]'
data['games'] = data['games'].apply(lambda x: ast.literal_eval(x))

# add image url 

In [41]:
data

,session_id,class,age,fileNameShortNew,filename,url,shuffler_ind,games
0,cdm_run_v41542395430636,a bear,age10,bear_sketch_age10_cdm_run_v41542395430636.png,bear_sketch_age10_cdm_run_v41542395430636.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,3798,[]
1,cdm_run_v41542396376182,a bear,age10,bear_sketch_age10_cdm_run_v41542396376182.png,bear_sketch_age10_cdm_run_v41542396376182.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,9823,[]
2,cdm_run_v41542494477217,a bear,age10,bear_sketch_age10_cdm_run_v41542494477217.png,bear_sketch_age10_cdm_run_v41542494477217.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,2660,[]
3,cdm_run_v41542572319870,a bear,age10,bear_sketch_age10_cdm_run_v41542572319870.png,bear_sketch_age10_cdm_run_v41542572319870.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,3543,[]
4,cdm_run_v41543101092847,a bear,age10,bear_sketch_age10_cdm_run_v41543101092847.png,bear_sketch_age10_cdm_run_v41543101092847.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,6075,[]
...,...,...,...,...,...,...,...,...
10208,cdm_run_v41548097908349,a train,age6,train_sketch_age6_cdm_run_v41548097908349.png,train_sketch_age6_cdm_run_v41548097908349.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,7725,[]
10209,cdm_run_v41548456052660,a train,age7,train_sketch_age7_cdm_run_v41548456052660.png,train_sketch_age7_cdm_run_v41548456052660.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,4919,[]
10210,cdm_run_v41549830251720,a train,age8,train_sketch_age8_cdm_run_v41549830251720.png,train_sketch_age8_cdm_run_v41549830251720.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,2351,[]
10211,cdm_run_v41550089564281,a train,age8,train_sketch_age8_cdm_run_v41550089564281.png,train_sketch_age8_cdm_run_v41550089564281.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,7259,[]


In [43]:
# for bucket in s3.buckets.all():
#     print(bucket.name)

#### Upload images to s3

In [45]:
runThis = 0
path_to_png = '/Users/brialong/Documents/GitHub/kiddraw/data/drawings/cdm_run_v4/filtered_dataset'
bucket_name = 'kiddraw'
if runThis:
    conn = boto3.resource('s3')
    b = conn.Bucket(bucket_name) 
    for ind,im in enumerate(os.listdir(path_to_png)):
        if im[-3:]=='png':
            print (ind, im)
            img_obj = b.Object(im)
            img_obj.upload_file(os.path.join(path_to_png, im))
            img_obj.Acl().put(ACL='public-read')
            clear_output(wait=True)  
# print 'Done!'

In [47]:
data['url']

0        https://kiddraw.s3.amazonaws.com/bear_sketch_a...
1        https://kiddraw.s3.amazonaws.com/bear_sketch_a...
2        https://kiddraw.s3.amazonaws.com/bear_sketch_a...
3        https://kiddraw.s3.amazonaws.com/bear_sketch_a...
4        https://kiddraw.s3.amazonaws.com/bear_sketch_a...
                               ...                        
10208    https://kiddraw.s3.amazonaws.com/train_sketch_...
10209    https://kiddraw.s3.amazonaws.com/train_sketch_...
10210    https://kiddraw.s3.amazonaws.com/train_sketch_...
10211    https://kiddraw.s3.amazonaws.com/train_sketch_...
10212    https://kiddraw.s3.amazonaws.com/train_sketch_...
Name: url, Length: 10213, dtype: object

In [44]:
# for key in b.objects.all():
#     print key.key


#### Convert the csv file to json and upload to mongoDB

In [48]:
# convert the csv file to json
import json
datajs = data.to_json(orient='records')
datajs = json.loads(datajs)

In [49]:
datajs

[{'session_id': 'cdm_run_v41542395430636',
  'class': 'a bear',
  'age': 'age10',
  'fileNameShortNew': 'bear_sketch_age10_cdm_run_v41542395430636.png',
  'filename': 'bear_sketch_age10_cdm_run_v41542395430636.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bear_sketch_age10_cdm_run_v41542395430636.png',
  'shuffler_ind': 3798,
  'games': []},
 {'session_id': 'cdm_run_v41542396376182',
  'class': 'a bear',
  'age': 'age10',
  'fileNameShortNew': 'bear_sketch_age10_cdm_run_v41542396376182.png',
  'filename': 'bear_sketch_age10_cdm_run_v41542396376182.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bear_sketch_age10_cdm_run_v41542396376182.png',
  'shuffler_ind': 9823,
  'games': []},
 {'session_id': 'cdm_run_v41542494477217',
  'class': 'a bear',
  'age': 'age10',
  'fileNameShortNew': 'bear_sketch_age10_cdm_run_v41542494477217.png',
  'filename': 'bear_sketch_age10_cdm_run_v41542494477217.png',
  'url': 'https://kiddraw.s3.amazonaws.com/bear_sketch_age10_cdm_run_v41542494477217.png',

In [52]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the stanford-cogsci.org user
pswd = auth.values[0][0]

# have to fix this to be able to analyze from local
conn = pm.MongoClient('mongodb://stanford:' + pswd + '@127.0.0.1')
db = conn['kiddraw']
coll = db['check_invalid_cdm_run_v4'] 

In [57]:
data

,session_id,class,age,fileNameShortNew,filename,url,shuffler_ind,games
0,cdm_run_v41542395430636,a bear,age10,bear_sketch_age10_cdm_run_v41542395430636.png,bear_sketch_age10_cdm_run_v41542395430636.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,3798,[]
1,cdm_run_v41542396376182,a bear,age10,bear_sketch_age10_cdm_run_v41542396376182.png,bear_sketch_age10_cdm_run_v41542396376182.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,9823,[]
2,cdm_run_v41542494477217,a bear,age10,bear_sketch_age10_cdm_run_v41542494477217.png,bear_sketch_age10_cdm_run_v41542494477217.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,2660,[]
3,cdm_run_v41542572319870,a bear,age10,bear_sketch_age10_cdm_run_v41542572319870.png,bear_sketch_age10_cdm_run_v41542572319870.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,3543,[]
4,cdm_run_v41543101092847,a bear,age10,bear_sketch_age10_cdm_run_v41543101092847.png,bear_sketch_age10_cdm_run_v41543101092847.png,https://kiddraw.s3.amazonaws.com/bear_sketch_a...,6075,[]
...,...,...,...,...,...,...,...,...
10208,cdm_run_v41548097908349,a train,age6,train_sketch_age6_cdm_run_v41548097908349.png,train_sketch_age6_cdm_run_v41548097908349.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,7725,[]
10209,cdm_run_v41548456052660,a train,age7,train_sketch_age7_cdm_run_v41548456052660.png,train_sketch_age7_cdm_run_v41548456052660.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,4919,[]
10210,cdm_run_v41549830251720,a train,age8,train_sketch_age8_cdm_run_v41549830251720.png,train_sketch_age8_cdm_run_v41549830251720.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,2351,[]
10211,cdm_run_v41550089564281,a train,age8,train_sketch_age8_cdm_run_v41550089564281.png,train_sketch_age8_cdm_run_v41550089564281.png,https://kiddraw.s3.amazonaws.com/train_sketch_...,7259,[]


In [53]:
# drop past data
reallyRun = 0
if reallyRun:
    coll.drop()
# print 'Currently there are {} records in this collection'.format(coll.count())

In [55]:
# upload data to the development collection
reallyRun = 1
if reallyRun:
    for i,_z in enumerate(datajs):
        coll.insert(_z)
        print('Inserted {} records.'.format(i))
        clear_output(wait=True)
else:
    print('Did not insert any new data.')

Inserted 10212 records.


In [60]:
## check how many records have been retrieved on the mturk study
# print 'Currently there are {} records in the collection'.format(coll.count())
#{"games": {"$size": 1}}
# {"_id": ObjectId("5e6a7e7d1943e62410f1ebb3")}
a = coll.find()
numGames = []
for rec in a:
    if len(rec['games']) > 0:
        numGames.append(len(rec['games'])) 
print (len(numGames))

0
